In [1]:
from sage.symbolic.expression import Expression
from delierium.MatrixOrder import Context, Mlex, Mgrlex, Mgrevlex
from IPython.display import Math
from dataclasses import dataclass
from delierium.helpers import latexer
from sage.symbolic.expression_conversions import ExpressionTreeWalker
from IPython.core.debugger import set_trace
import doctest
from typing import Union, Any, Optional, TYPE_CHECKING, cast
from functools import partial, wraps, cache
from operator import mul
from IPython.display import Math

In [2]:
x, y= var("x y")
xi  = function("xi", latex_name="\\xi")(x,y)
eta = function("eta", latex_name = "\\eta")(x,y)
ctx = Context([xi, eta], [x, y], Mgrevlex)
xi_x = diff(xi, x)
xi_y = diff(xi, y)
xi_xx = diff(xi, x,x)
xi_yy = diff(xi, y,y)
xi_xy = diff(xi, x,y)
eta_x = diff(eta, x)
eta_y = diff(eta, y)
eta_xx = diff(eta, x,x)
eta_yy = diff(eta, y,y)
eta_xy = diff(eta, x,y)

A = function("A")(x,y)
B = function("B")(x,y)
C = function("C")(x,y)
D = function("D")(x,y)
A_x = diff(A, x)
A_y = diff(A, y)
B_x = diff(B, x)
B_y = diff(B, y)
C_x = diff(C, x)
C_y = diff(C, y)
D_x = diff(D, x)
D_y = diff(D, y)

In [3]:
e1 = xi_yy - 2*A*eta_y - B*xi_y + A*xi_x - A_y*eta-A_x*xi
e2 = eta_xx - D*eta_y +C*eta_x +2*D*xi_x +D_y*eta+D_x*xi
e3 = eta_xy - xi_xx/2 +B*eta_x+3*D*xi_y/2+C*xi_x/2 + C_y*eta/2 + C_x*xi/2
e4 = eta_yy - 2*xi_xy +B*eta_y + 3*A*eta_x + 2*C*xi_y + B_y*eta +B_x*xi

In [4]:
from functools import partial, wraps, cache
from operator import mul
#def contextualize(func):
#    @wraps(function)
#    def context_wrapper(context, *args):
#        return partial(func, context)
#    return context_wrapper

In [5]:
_Sage_Expression = sage.symbolic.expression.Expression
_Delierium_Context = Context
def gen_summands(context: _Delierium_Context, expression: _Sage_Expression) -> list[_Sage_Expression]:
    try :
        if expression.operator().__name__ == 'add_vararg':
            return expression.operands()
    except AttributeError:
        return [expression]
    return [expression]

In [6]:
class DelieriumNotALinearPDE(TypeError):
    def __init__ (self, expression):
        #self.message = f"{expression=} is not a valid linear PDE"
        super().__init__(f"{expression=} is not a valid term in linear PDE")

@dataclass
class DT:
    context: Context
    expression: _Sage_Expression
    coefficient: _Sage_Expression
    derivative: _Sage_Expression
    function: _Sage_Expression
    order: tuple[int]
    comparison_order: tuple[int]
    def __lt__ (self, other):
        return self.context.gt(self.comparison_order,other.comparison_order)


In [7]:
def is_derivative(e: Any) -> bool:
    """
    Check if an expression is a pure derivative
    >>> x, y= var("x y")
    >>> xi  = function("xi")(x,y)
    >>> ctx = Context([xi], [x,y])
    >>> xi_x = diff(xi, x)
    >>> xi_y = diff(xi, y)
    >>> xi_xx = diff(xi, x,x)
    >>> xi_yy = diff(xi, y,y)
    >>> xi_xy = diff(xi, x,y)
    >>> A = function("A")(x,y)
    >>> A_x = diff(A, x)
    >>> is_derivative(A)
    False
    >>> is_derivative(1)
    False
    >>> is_derivative(A*xi_x)
    False
    >>> is_derivative(xi)
    False
    >>> is_derivative(xi_xy)
    True
    >>> is_derivative(2*xi_x)
    False
    """
    try:
        return e.operator().__class__.__name__ == "FDerivativeOperator"
    except AttributeError:
        return False

doctest.testmod()

/home/tapir/mambaforge/envs/sage/lib/python3.11/inspect.py:753: DeprecationWarning: 
Importing Euler_Phi from here is deprecated; please use "from sage.arith.misc import Euler_Phi" instead.
See https://github.com/sagemath/sage/issues/30322 for details.
  return hasattr(f, '__wrapped__')
/home/tapir/mambaforge/envs/sage/lib/python3.11/inspect.py:753: DeprecationWarning: this class is deprecated, do not use
See https://github.com/sagemath/sage/issues/31545 for details.
  return hasattr(f, '__wrapped__')
/home/tapir/mambaforge/envs/sage/lib/python3.11/inspect.py:753: DeprecationWarning: 
Importing LaurentSeries from here is deprecated; please use "from sage.rings.laurent_series_ring_element import LaurentSeries" instead.
See https://github.com/sagemath/sage/issues/33602 for details.
  return hasattr(f, '__wrapped__')
/home/tapir/mambaforge/envs/sage/lib/python3.11/inspect.py:753: DeprecationWarning: 
Importing Moebius from here is deprecated; please use "from sage.arith.misc import Moebiu

TestResults(failed=0, attempted=16)

In [8]:
def order_of_derivative(context: _Delierium_Context, e: Any):
    '''Returns the vector of the orders of a derivative respect to its variables
    >>> x, y, z= var("x y z")
    >>> f1  = function("f1")(x,y, z)
    >>> ctx = Context([f1], [x,y, z])
    >>> order_of_derivative(ctx, diff(f1, x, x, x, x))
    [4, 0, 0]
    >>> order_of_derivative(ctx, f1)    
    [0, 0, 0]
    >>> order_of_derivative(ctx, diff(f1, x, x, x, y, z, z))
    [3, 1, 2]
    >>> ctx = Context([f1], [z, x, y])
    >>> order_of_derivative(ctx, diff(f1, x, x, x, x))
    [4, 0, 0]
    >>> order_of_derivative(ctx, diff(f1, x, x, x, y, z, z))
    [3, 1, 2]
    '''
    res = [0] * len(e.variables())
    if not is_derivative(e):
        return res
    for variable in e.variables():
        # XXX. here seems to be the crucial part: order can depend on:
        # - either the order given by sage by
        # -- sage order
        # -- order given by the order given by function definition
        # - the order given by context
        i = context._independent.index(variable)
        res[i] = e.operator().parameter_set().count(i)
    return res
doctest.testmod()    

TestResults(failed=0, attempted=25)

In [9]:
def _compute_comparison_vector(context, func):
    """
    >>> x, y, z= var("x y z")
    >>> f1  = function("f1")(x, y, z)
    >>> f2  = function("f2")(x, y, z)
    >>> f3  = function("f3")(x, y, z)
    >>> ctx = Context([f1, f2, f3], [x, y, z])
    >>> _compute_comparison_vector(ctx, f1)
    [1, 0, 0]
    >>> _compute_comparison_vector(ctx, f3)
    [0, 0, 1]
    """
    iv = [0] * len(context._dependent)
    if func in context._dependent:
        iv[context._dependent.index(func)] = 1
    elif func.operator() in context._dependent:
        iv[context._dependent.index(func.operator())] = 1
    else:
        pass
    return iv    
    
doctest.testmod()        

TestResults(failed=0, attempted=32)

In [10]:
def compute_order(context, expression):
    """computes the monomial tuple from the derivative part"""
    if is_derivative(expression):
        return order_of_derivative(expression, context, len(context._independent))
    # XXX: Check can that be within a system of linead PDEs ? I think not, as the function must then be identically to zero
    return [0] * len(context._independent)

In [11]:
def _gen_dt(context: _Delierium_Context, expression: _Sage_Expression) -> tuple[tuple[_Sage_Expression], tuple[_Sage_Expression]]:
    coeff = []
    derivs = []
    match expression.operator().__class__.__name__:
        case "FDerivativeOperator":
            if expression.operator().function() in context._dependent:
                derivs.append(expression)
            else:
                coeff.append(expression)
        case "Expression":
            print("Symbolic Expression")
        case "NewSymbolicFunction":
            if expression.operator() in context._dependent:
                derivs.append(expression)
            else:
                coeff.append(expression)
        case "function":
            if expression.operator().__name__ == "mul_vararg":
                for operand in expression.operands():
                    if not operand.operator():
                        coeff.append(operand)
                    else:
                        c, d = _gen_dt(context, operand)
                        coeff.extend(c)
                        derivs.extend(d)
        case _:
            coeff.append(expression)
    return coeff, derivs

def gen_dt(context: _Delierium_Context, expression: _Sage_Expression)-> bool:
    c, d = _gen_dt(context, expression)
    if not d:
        raise DelieriumNotALinearPDE(expression)
    d = reduce(mul, d, 1)
    order = order_of_derivative(context, d)
    return DT(context,
              expression,
              reduce(mul, c, 1), d,
              d.operator().function() if is_derivative(d) else d.function(),              
              order,
              _compute_comparison_vector(context, d.operator().function() if is_derivative(d) else d.function()) + order
             )    

In [12]:
gen_dt(ctx, 2*A_x*B_x*xi_xy)
print("1"*20)
gen_dt(ctx, xi)
print("2"*20)
gen_dt(ctx, 2*xi)
print("3"*20)
#gen_dt(ctx, A)
#print("4"*20)
#gen_dt(ctx, 2*A*D)
#print("5"*20)
gen_dt(ctx, xi_x)
print("4"*20)
gen_dt(ctx, A*xi_x)



11111111111111111111
22222222222222222222
33333333333333333333
44444444444444444444


DT(context=<delierium.MatrixOrder.Context object at 0x7f2200fe3400>, expression=A(x, y)*diff(xi(x, y), x), coefficient=A(x, y), derivative=diff(xi(x, y), x), function=xi, order=[1, 0], comparison_order=[1, 0, 1, 0])

In [13]:
gen_dt(ctx, 2*A_x*B_x*xi_xy)

DT(context=<delierium.MatrixOrder.Context object at 0x7f2200fe3400>, expression=2*diff(A(x, y), x)*diff(B(x, y), x)*diff(xi(x, y), x, y), coefficient=2*diff(A(x, y), x)*diff(B(x, y), x), derivative=diff(xi(x, y), x, y), function=xi, order=[1, 1], comparison_order=[1, 0, 1, 1])

In [14]:
v = list(map(lambda _: gen_dt(ctx, _), gen_summands(ctx, e1)))

In [15]:
class DP(_Sage_Expression):
    def __init__(self, context, expression):
        self.context = context
        self.expression = expression
        self.dts = list(sorted(map(lambda _: gen_dt(context, _), gen_summands(context, expression))))
        lcoeff = self.dts[0].coefficient
        for dt in self.dts:
            dt.coefficient /= lcoeff
        self.lder = self.dts[0].derivative
        self.comp = self.dts[0].comparison_order

    def __lt__ (self, other):
        return self.context.gt(self.comp, other.comp)


In [16]:
dp=DP(ctx, e1)

In [17]:
dp.lder

diff(xi(x, y), x)

In [18]:
for _ in dp.dts:
    print(_.coefficient, _.derivative, _.comparison_order)


1 diff(xi(x, y), x) [1, 0, 1, 0]
1/A(x, y) diff(xi(x, y), y, y) [1, 0, 0, 2]
-B(x, y)/A(x, y) diff(xi(x, y), y) [1, 0, 0, 1]
-2 diff(eta(x, y), y) [0, 1, 0, 1]
-diff(A(x, y), x)/A(x, y) xi(x, y) [1, 0, 0, 0]
-diff(A(x, y), y)/A(x, y) eta(x, y) [0, 1, 0, 0]


In [19]:
dps=[DP(ctx, _) for _ in [e4,e2,e1,e3]]
for _ in dps:
    print(_.lder)

diff(xi(x, y), x, y)
diff(eta(x, y), x, x)
diff(xi(x, y), x)
diff(xi(x, y), x, x)


In [20]:
def _Janet_Basis(dps):
    return sorted(dps)

In [21]:
jb=_Janet_Basis(dps)

In [22]:
def latexer_dt(context, dt):
    def _latex_derivative(context, deriv):
        if is_derivative(deriv):
            func = deriv.function().operator().function()._latex_()
            ps = deriv.operator().parameter_set()
            variables = deriv.operands()
            sub = ",".join(map(lambda _ : variables[_]._latex_(), ps))
            return r"%s_{%s}" % (func, sub)
        else:
            return deriv.function().operator()._latex_()
    def _latex_coeff(context, coeff):
        if str(coeff) == '1':
            return ""
        # need to be more fine granular
        return coeff._latex_()
        
    d = _latex_derivative(context, dt.derivative)
    c = _latex_coeff(context, dt.coefficient)
    return f"{c} {d}"
    

latexer_dt(ctx, gen_dt(ctx, (-4*B_x/A_x)*xi_xy))

'-\\frac{4 \\, \\frac{\\partial}{\\partial x}B\\left(x, y\\right)}{\\frac{\\partial}{\\partial x}A\\left(x, y\\right)} \\xi_{x,y}'

In [23]:
def latexter_dp(context , dp):
    return "+".join(map(lambda _ : latexer_dt(context, _), dp.dts)).replace("+-", "-")

In [24]:
latexter_dp(ctx, dps[2])

' \\xi_{x}+\\frac{1}{A\\left(x, y\\right)} \\xi_{y,y}-\\frac{B\\left(x, y\\right)}{A\\left(x, y\\right)} \\xi_{y}-2 \\eta_{y}-\\frac{\\frac{\\partial}{\\partial x}A\\left(x, y\\right)}{A\\left(x, y\\right)} \\xi-\\frac{\\frac{\\partial}{\\partial y}A\\left(x, y\\right)}{A\\left(x, y\\right)} \\eta'

In [25]:
Math(latexter_dp(ctx, dps[3]))

<IPython.core.display.Math object>

In [41]:
class Janet_Basis:
    def __init__(self, context: Context, expressions: list[_Sage_Expression]):
        set_trace()
        self.dps = [DP(context, _) for _ in expressions]

In [42]:
jb = Janet_Basis(ctx, [e1,e2,e3,e4])

> /tmp/ipykernel_43754/115573397.py(4)__init__()
      1 class Janet_Basis:
      2     def __init__(self, context: Context, expressions: list[_Sage_Expression]):
      3         set_trace()
----> 4         self.dps = [DP(context, _) for _ in expressions]



ipdb>  s


--Call--
> /tmp/ipykernel_43754/115573397.py(4)<listcomp>()
      1 class Janet_Basis:
      2     def __init__(self, context: Context, expressions: list[_Sage_Expression]):
      3         set_trace()
----> 4         self.dps = [DP(context, _) for _ in expressions]



ipdb>  s


> /tmp/ipykernel_43754/115573397.py(4)<listcomp>()
      1 class Janet_Basis:
      2     def __init__(self, context: Context, expressions: list[_Sage_Expression]):
      3         set_trace()
----> 4         self.dps = [DP(context, _) for _ in expressions]



...

ipdb>  n


--Return--
None
> /tmp/ipykernel_43754/1236637525.py(10)__init__()
      8             dt.coefficient /= lcoeff
      9         self.lder = self.dts[Integer(0)].derivative
---> 10         self.comp = self.dts[Integer(0)].comparison_order
     11 
     12     def __lt__ (self, other):



ipdb>  n


> /tmp/ipykernel_43754/115573397.py(4)<listcomp>()
      1 class Janet_Basis:
      2     def __init__(self, context: Context, expressions: list[_Sage_Expression]):
      3         set_trace()
----> 4         self.dps = [DP(context, _) for _ in expressions]



ipdb>  c


In [43]:
for _ in jb.dps:
    print(_.__class__)
    Math(latexter_dp(ctx, _))

<class '__main__.DP'>
<class '__main__.DP'>
<class '__main__.DP'>
<class '__main__.DP'>
